#### Defining llm & embedding

In [4]:
import pandas as pd
import numpy as np
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.langchain.proxy import OpenAIEmbeddings
from ipywidgets import widgets

llm_model_name = widgets.Dropdown(
    options=[
        "gpt-35-turbo",
        "gpt-35-turbo-16k",
        "gpt-4",
        "gpt-4-32k",
        "gpt-4-turbo",
        "gemini-1.0-pro",
        "gpt-4-vision"
        # "tiiuae--falcon-40b-instruct"
    ],
    value="gpt-35-turbo-16k",
    description="LLM Model Name",
    disabled=False,
)


llm = ChatOpenAI(proxy_model_name=llm_model_name.value)
embeddings = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.langchain import LangChainLLM

llama_llm = LangChainLLM(llm)
llama_emb= LangchainEmbedding(embeddings)

from llama_index.core import Settings
Settings.embed_model = embeddings
Settings.llm = llm

C:\Users\I068117\AppData\Local\Temp\ipykernel_35900\2491488312.py:3: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  from llm_commons.langchain.proxy import ChatOpenAI


### Creating LLama_index documents from the corpus data -US & Global Flex policy


In [6]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=llama_emb
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512)

In [1]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["202204_Pledge_to_Flex_Global_FAQ.pdf"]).load_data()
US_document= SimpleDirectoryReader(input_files=["US Flex Work FAQs.pdf"]).load_data()

Ignoring wrong pointing object 93 0 (offset 0)


In [7]:
nodes = splitter.get_nodes_from_documents(documents)
US_nodes = splitter.get_nodes_from_documents(US_document)

#### Strict meta-data filtering can be between HR policy document vs Concur travel policy document. 
#### Non-strict meta data :  local vs global HR policy document. 

In [38]:
nodes_list = []
for node in nodes:
    node.metadata["context"] = "global"
    nodes_list.append(node)

for node in US_nodes:
    node.metadata["context"] = "US"
    nodes_list.append(node)

In [40]:
print(nodes_list[50].get_content(metadata_mode="all"))

page_label: 6
file_name: US Flex Work FAQs.pdf
file_path: US Flex Work FAQs.pdf
file_type: application/pdf
file_size: 627129
creation_date: 2024-07-07
last_modified_date: 2024-07-07
context: US

Do not use the Full 
Home Office setting for this situation.  
Why was a distance of  greater than 40 miles selected as a marker for Full Home Office  or Hybrid - Majority Home Office  
situations to remain if business -approved ? 
SAP does not define remote work arrangements in contractual agreement s for employees in  the US or Canada . All 
employees are assigned to an SAP office . This meant  a uniform benchmark was required  to define employees 
working from locations where no SAP office exists. Generally, 50 miles is the industry standard being used by 
many companies to define  commutable distance. SAP elected for the benefit of employees to lower this  
benchmark  to 40 miles.  
How is it confirmed who remains Full Home Office or Hybrid - Majority Home Office  status in Success Map due 

##### ChromaVector store

In [28]:
import nest_asyncio
nest_asyncio.apply()

In [29]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [30]:
import chromadb
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [31]:
from llama_index.vector_stores.chroma import ChromaVectorStore

#Storing index on the disk
from llama_index.core import (
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

### Strict Meta-data filtering based on global policy document [Optional]

In [75]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

filters = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="202204_Pledge_to_Flex_Global_FAQ.pdf"),
    ]
)

index= VectorStoreIndex(nodes_list,storage_context,embed_model=llama_emb)
#context[0].node.text

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.a

HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.a

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.a

HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.a

[['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['12', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['10', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global']]

In [56]:
query="I am facing personal circumstances that might make it challenging for me to follow the evolved hybrid work guidance. What kind of exceptions are available? "
retriever = index.as_retriever(filters=filters, similarity_top_k=10)
context=retriever.retrieve(query)
ci=[]
for c in context:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"


[['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['12', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['10', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global']]

In [53]:
context[0].node.text

'Exceptions  3.1 I am facing personal circumstances that might make it challenging for me to follow the evolved hybrid work guidance. What kind of exceptions are available?  Rooted in our commitment to align business, customer, and employee needs, we are empowering managers to find the right solutions together with their teams, supported by defined guardrails. At a global policy level, employees may be partially or fully exempt from the three-days-a-week in-office requirement for the following scenarios:  • Existing legally binding agreements remain valid, e.g., employees contractually entitled to work remotely for more than two days a week.  • Part-time employees can adjust their in-office days proportionally to their contract hours. • Employees requiring a different setup because of permanent specific needs, e.g., disability, neurodivergent conditions, or long-term health / medical conditions, will be accommodated.  • Managers may grant temporary exceptions for personal reasons such 

###  Baseline - Assuming no strict filtering for this use case as both are HR policy document

In [76]:
retriever = index.as_retriever(similarity_top_k=10)

context=retriever.retrieve(query)
ci=[]
for c in context:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci
#context[2].node.text

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"


[['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', 'US Flex Work FAQs.pdf', 'US']]

### 2 US chunks are in the top 10 and the first one is at the 3rd place!!

## ADR#1  Non-Strict Meta-data [US | Global] filtering using embedding search

In this approach the query along with the nonStrictMetadataFilter is embedded followed by an Embedding Search.

The stored embeddings in the Vector database represent the chunk along with the metadata.- Metadata is embedded along with the document

###### Embedding document & non strict meta-data 

In [83]:
from llama_index.core import Document
Lama_doc=[]

for node in nodes_list:
    non_strict_metadata = node.metadata["context"]
    Lama_doc.append(Document(text=node.text+non_strict_metadata,metadata = node.metadata))

In [103]:
from llama_index.core.node_parser import SentenceSplitter
node_parser = SentenceSplitter()
nodes_meta=node_parser.get_nodes_from_documents(Lama_doc)
print(nodes_meta[10].get_content(metadata_mode="all"))

page_label: 4
file_name: 202204_Pledge_to_Flex_Global_FAQ.pdf
file_path: 202204_Pledge_to_Flex_Global_FAQ.pdf
file_type: application/pdf
file_size: 578756
creation_date: 2024-07-07
last_modified_date: 2024-07-07
context: global

See Legal Notice on www.sap.com/legal-notice for use terms, disclaimers, disclosures, or restrictions related to this material. 4 / 12 Based on your personal working preferences, you can of course also work more than three days per week from the oﬃce depending on local circumstances. For more information about ﬂex work in your location, please have a look at your local portal page.  1.4 Are there any exceptions? While the new evolved hybrid work guidance applies to all employees globally, we acknowledge that certain situations may require temporary exceptions. In line with our global exception framework, the locally managed evolved hybrid work model offers a practical, people-centered exception handling considering business requirements, local labor law, polici

In [123]:
index_metadata = VectorStoreIndex(nodes_meta,storage_context,embed_model=llama_emb)

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.a

HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.a

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.a

HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.a

#### Embedding query & non-strict meta-data (in this case, adding 'US')

In [124]:
query_US="I am facing personal circumstances that might make it challenging for me to follow the evolved hybrid work guidance. What kind of exceptions are available in the US?"

retriever = index_metadata.as_retriever(similarity_top_k=10)

ADR2_context=retriever.retrieve(query_US)
ci=[]
for c in ADR2_context:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"


[['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', 'US Flex Work FAQs.pdf', 'US']]

### 3 US chunks are in the top 10 !!

## ADR#2  Non-strict meta data filtering using reranker model

In this approach after the embedding search has been done the query along with the nonStrictMetadataFilter is passed to the re-ranker model which in it's reranking should be able to consider the filter.

#### Without Re-ranker - BASELINE SCENARIO

In [112]:
retriever = index.as_retriever(similarity_top_k=10)
context=retriever.retrieve(query)
ci=[]
for c in context:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"


[['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', 'US Flex Work FAQs.pdf', 'US']]

#### With Cohere Re-ranking and  non-strict meta-data filter by adding it to the query - introducing bias for US

In [111]:
import cohere
# init client
co = cohere.Client("fO0pe0IbZyWcHJ2HLhLb1Zc9b0HbFb7bYjpsA2On")
context_doc = []
# Assuming context is a list of objects that have the method get_content(metadata_mode="all")
for i in range(len(context)):
    context_doc.append(context[i].get_content(metadata_mode="all"))
    
rerank_docs = co.rerank(
    query=query_US, documents=context_doc, top_n=10, model="rerank-english-v2.0"
)
cohere_ranked_nodes= [context[i.index] for i in rerank_docs.results]
ci=[]
for c in cohere_ranked_nodes:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


[['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', 'US Flex Work FAQs.pdf', 'US'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global']]

#### Re-Ranker with non-strict meta data after embedding search( ADR 2)

In [125]:
context_doc = []
# Assuming context is a list of objects that have the method get_content(metadata_mode="all")

for i in range(len(ADR2_context)):
    context_doc.append(ADR2_context[i].get_content(metadata_mode="all"))
    
rerank_docs = co.rerank(
    query=query_US, documents=context_doc, top_n=10, model="rerank-english-v2.0"
)
cohere_ranked_nodes= [ADR2_context[i.index] for i in rerank_docs.results]
ci=[]
for c in cohere_ranked_nodes:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


[['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', 'US Flex Work FAQs.pdf', 'US'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global']]

### Re-ranker with meta-filtering ranks has very minimal effect in the overall ranking

## ADR#3: Non-Strict Metadata Filtering using Custom Scoring

#### Baseline

In [117]:
retriever = index.as_retriever(similarity_top_k=10)
context=retriever.retrieve(query)
ci=[]
for c in context:
    ci.append([c.node.metadata['page_label'],c.node.metadata['file_name'],c.node.metadata['context']])
ci

INFO:httpx:HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcf97822fc795ac1/embeddings?api-version=2023-09-01-preview "HTTP/1.1 200 OK"


[['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['3', 'US Flex Work FAQs.pdf', 'US'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['7', 'US Flex Work FAQs.pdf', 'US']]

 Re-rank the retrieved documents based on how close the document metadata is to query metadata 1.3. Return documents based off final ranking (limit to most relevant subset)

In [121]:
Query_metadata='US'
# Custom sorting function to sort based on the query & doc meta-data
ci_sorted = sorted(ci, key=lambda x: (x[2] != Query_metadata, x[2]))
ci_sorted

[['3', 'US Flex Work FAQs.pdf', 'US'],
 ['7', 'US Flex Work FAQs.pdf', 'US'],
 ['7', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['2', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['8', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['9', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['4', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global'],
 ['1', '202204_Pledge_to_Flex_Global_FAQ.pdf', 'global']]

#### Reciprocal Fusion re-ranking model

In [ ]:
from llama_index.core.retrievers import BaseRetriever

## define meta-data retriver

class metadataRetriever(BaseRetriever):
    def __init__(self):
    
        super().__init__()

    def _retrieve(self, query, **kwargs):
        
        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

metadata_retriever = metadataRetriever()

In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [retriever, metadata_retriever],
    similarity_top_k=10,
    num_queries=1,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

### Meta- data extractors from the document

In [152]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)

from llama_index.core.schema import MetadataMode

from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

extractors_1 = [
    QuestionsAnsweredExtractor(
        questions=3, llm=llama_llm, metadata_mode=MetadataMode.EMBED
    ),
]

class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": "Cluster1"
            }
            for node in nodes
        ]
        return metadata_list

In [19]:
from llama_index.core.ingestion import IngestionPipeline

# process nodes with metadata extractors
pipeline = IngestionPipeline(transformations=[node_parser, *CustomExtractor])

nodes_1 = pipeline.run(nodes=nodes[:3], in_place=False, show_progress=True)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\I068117\Anaconda3\envs\myenv10\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.35s/it]


### Auto retreival filtering using LLM 

In [126]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo

vector_store_info = VectorStoreInfo(
    content_info="Scifact",
    metadata_info=[
        MetadataInfo(
            name="idx",
            type="str",
            description=(
                "it's number index in the metadata"
            ),
        ),
    ],
)

retriever = VectorIndexAutoRetriever(
    index, vector_store_info=vector_store_info,similarity_top_k=10)